Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import draw_shapes
import preprocessing

# cuda on Nvidia, mps on Mac
device = torch.device("cuda:0" if torch.cuda.is_available() else "mps")

Setup Tensorboard

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('logs/introduction')

%load_ext tensorboard
%tensorboard --logdir ./ --port 6006

Test preprocessing

In [ ]:
#draw circle and get surface pixels
circle = draw_shapes.draw_circle()
image = preprocessing.binary_to_image(circle)
surface = preprocessing.get_surface_pixels(circle)
surface_image = preprocessing.binary_to_image(surface)
#plot both images
fig, ax = plt.subplots(1, 2)
ax[0].imshow(image, cmap='gray')
ax[0].set_title('Circle')
ax[1].imshow(surface_image, cmap='gray')
ax[1].set_title('Surface')
plt.show()
#now for donut
donut = draw_shapes.draw_donut()
image = preprocessing.binary_to_image(donut)
surface = preprocessing.get_surface_pixels(donut)
surface_image = preprocessing.binary_to_image(surface)
#plot both images
fig, ax = plt.subplots(1, 2)
ax[0].imshow(image, cmap='gray')
ax[0].set_title('Donut')
ax[1].imshow(surface_image, cmap='gray')
ax[1].set_title('Surface')
plt.show()
#now for spriale
spiral = draw_shapes.draw_spiral()
image = preprocessing.binary_to_image(spiral)
surface = preprocessing.get_surface_pixels(spiral)
surface_image = preprocessing.binary_to_image(surface)
#plot both images
fig, ax = plt.subplots(1, 2)
ax[0].imshow(image, cmap='gray')
ax[0].set_title('Spiral')
ax[1].imshow(surface_image, cmap='gray')
ax[1].set_title('Surface')
plt.show()
